In [ ]:
!pip install roboflow


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="key")
project = rf.workspace("sliit-kuemd").project("cattle-diseases")
version = project.version(2)
dataset = version.download("multiclass")

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoImageProcessor
from torch.utils.data import Dataset
from transformers import AutoImageProcessor
from PIL import Image
from torchvision import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments
from torch import nn, optim

In [ ]:


# Load and merge labels from both train and test folders
def load_and_merge_labels(folder_paths):
    merged_df = pd.DataFrame()
    for path in folder_paths:
        csv_path = os.path.join(path, "_classes.csv")
        df = pd.read_csv(csv_path)
        df["image_path"] = df.iloc[:, 0].apply(lambda x: os.path.join(path, x))
        merged_df = pd.concat([merged_df, df], ignore_index=True)
    return merged_df



In [ ]:

# Merge train + test
train_df = load_and_merge_labels([
    "/content/cattle-diseases-2/train",
    "/content/cattle-diseases-2/test"
])
valid_df = load_and_merge_labels(["/content/cattle-diseases-2/valid"])



In [ ]:
# Show sample data
train_df.head()


,filename,(BRD),Bovine,Contagious,Dermatitis,Disease,Ecthym,Respiratory,Unlabeled,healthy,lumpy,skin,image_path
0,37_jpg.rf.583270f81aad69fe1fa331edd1a8d7da.jpg,0,0,0,0,0,0,0,0,0,1,1,/content/cattle-diseases-2/train/37_jpg.rf.583...
1,8_jpg.rf.5744e632fa0e1f44d677bc5405233f02.jpg,0,0,0,0,0,0,0,0,0,1,1,/content/cattle-diseases-2/train/8_jpg.rf.5744...
2,36_jpg.rf.530ca130a108b3e31c4f812330bd4211.jpg,0,0,0,0,0,0,0,0,0,1,1,/content/cattle-diseases-2/train/36_jpg.rf.530...
3,Ayrshirecattle30_c_jpg.rf.569ea435b44be8110b7c...,0,0,0,0,0,0,0,0,1,0,0,/content/cattle-diseases-2/train/Ayrshirecattl...
4,Ayrshirecattle146_c_jpg.rf.552c1cf35f85ae054c3...,0,0,0,0,0,0,0,0,1,0,0,/content/cattle-diseases-2/train/Ayrshirecattl...


In [ ]:
valid_df.head()

,filename,(BRD),Bovine,Contagious,Dermatitis,Disease,Ecthym,Respiratory,Unlabeled,healthy,lumpy,skin,image_path
0,istockphoto-1319281522-612x612_jpg.rf.995226a1...,0,0,0,0,0,0,0,0,1,0,0,/content/cattle-diseases-2/valid/istockphoto-1...
1,IMG_20220829_124626_jpg.rf.a354942b646265839d9...,0,0,0,0,0,0,0,0,0,1,1,/content/cattle-diseases-2/valid/IMG_20220829_...
2,images_jpeg.rf.a5131ece4904d8dcaf77e31b7258378...,1,1,0,0,1,0,1,0,0,0,0,/content/cattle-diseases-2/valid/images_jpeg.r...
3,IMG-20220830-WA0065_jpg.rf.9b091d6fbdde75a3d8e...,0,0,0,0,0,0,0,0,0,1,1,/content/cattle-diseases-2/valid/IMG-20220830-...
4,Ayrshirecattle69_jpg.rf.983d9add00cdf1d9e36b3d...,0,0,0,0,0,0,0,0,1,0,0,/content/cattle-diseases-2/valid/Ayrshirecattl...


In [ ]:
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [ ]:
class CowDiseaseDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = row['image_path']
        labels = torch.tensor(row.values[1:-1], dtype=torch.float32)  # Exclude image path column and last label column

        # Load the image
        image = Image.open(image_path).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, labels

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Zoom in
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Match ViT expectations
])

In [ ]:
# Create datasets for training and validation
train_dataset = CowDiseaseDataset(dataframe=train_df, transform=train_transform)
valid_dataset = CowDiseaseDataset(dataframe=valid_df, transform=train_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

In [ ]:
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=11,  # Number of classes
    problem_type="multi_label_classification"  # Multi-label classification
)

# Set device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # For multi-label classification
optimizer = optim.Adam(model.parameters(), lr=2e-5)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",  # Evaluation at the end of every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir='./logs',  # For storing logs
    logging_steps=10,  # Log every 10 steps
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(10):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(pixel_values=images).logits
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


Epoch 1, Loss: 0.2479
Epoch 2, Loss: 0.2287
Epoch 3, Loss: 0.2126
Epoch 4, Loss: 0.1980
Epoch 5, Loss: 0.1845
Epoch 6, Loss: 0.1721
Epoch 7, Loss: 0.1607
Epoch 8, Loss: 0.1501
Epoch 9, Loss: 0.1403
Epoch 10, Loss: 0.1313


In [ ]:
from transformers import AutoImageProcessor
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
class_names = [
    "BRD",           # Bovine Respiratory Disease
    "Bovine",
    "Contagious",
    "Dermatitis",
    "Disease",
    "Ecthym",
    "Respiratory",
    "Unlabeled",
    "Healthy",
    "Lumpy",
    "Skin"
]


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [ ]:
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.sigmoid(logits).squeeze()  # Use sigmoid for multi-label

    predicted_labels = (probs > 0.5).int().cpu().numpy()

    print(f"\n📷 Results for: {image_path}")
    for idx, label in enumerate(predicted_labels):
        status = "✅" if label == 1 else "❌"
        print(f"{class_names[idx]}: {status} ({probs[idx]:.2f})")


In [ ]:
predict_image("/content/lumpy.png")



📷 Results for: /content/lumpy.png
BRD: ❌ (0.08)
Bovine: ❌ (0.07)
Contagious: ❌ (0.08)
Dermatitis: ❌ (0.08)
Disease: ❌ (0.08)
Ecthym: ❌ (0.07)
Respiratory: ❌ (0.07)
Unlabeled: ❌ (0.07)
Healthy: ❌ (0.41)
Lumpy: ✅ (0.61)
Skin: ✅ (0.62)


In [ ]:
predict_image("/content/brd.jpg")


📷 Results for: /content/brd.jpg
BRD: ❌ (0.11)
Bovine: ❌ (0.09)
Contagious: ❌ (0.11)
Dermatitis: ❌ (0.10)
Disease: ❌ (0.10)
Ecthym: ❌ (0.09)
Respiratory: ❌ (0.10)
Unlabeled: ❌ (0.09)
Healthy: ✅ (0.83)
Lumpy: ❌ (0.16)
Skin: ❌ (0.19)
